## Convert from XLSX -> Pandas dataframe

In [110]:
from openpyxl import load_workbook
from pandas import DataFrame
import json
import subprocess

import numpy as np
from openpyxl import load_workbook
from pandas import DataFrame


data = load_workbook('laws-influences.xlsx').active.values
header = next(data)
df = DataFrame(data, columns=header)
df['Origin ID'] = df['Origin ID'].astype("Int64")
# df.fillna(value=MISSING_VALUE, inplace=True)
# df = df.applymap(lambda v: v.strip() if type(v) is str else v)
df.set_index('Law ID', inplace=True, verify_integrity=True)


# Check for dangling nodes (no child & no parent)
roots = set(df[df['Origin ID'].isnull()].index)
leaf_nodes = set(df.index) - set(df['Origin ID'])
dangling = roots & leaf_nodes
if len(dangling) > 0:
    print(f"[WARNING] Found dangling nodes: {dangling}")

# Check for incorrect references
invalid_refs = set(df['Origin ID']) - {np.nan} - set(df.index)
if invalid_refs:
    print(f"[WARNING] Invalid references: {invalid_refs}")


[WARNING] Found dangling nodes: {33}


In [111]:
# How many laws repeated another one word for word?
# TODO Origin ID is missing entries
df[
    ~df['Law of Origin'].isnull() 
].groupby('Connection with Previous Law').aggregate(np.size)['Origin ID']

Connection with Previous Law
Explicit Duplication     9
Identical                7
Similar Content         31
Name: Origin ID, dtype: Int64

In [112]:
# How many laws repeated another one word for word?
df[
    df['Origin ID'].notnull()
].groupby('Connection with Previous Law').aggregate(np.size)['Origin ID']

Connection with Previous Law
Explicit Duplication     9
Identical                7
Similar Content         32
Name: Origin ID, dtype: Int64

In [92]:
# How many connections crossed the Indian Ocean/Atlantic boundary?
df[~df['Origin ID'].isnull()].join(
    df, on='Origin ID', rsuffix="_origin"
).groupby(['Location_origin', 'Location']).aggregate(np.size)['Name of the Law']

Location_origin                Location                     
Caribbean                      Guyana                           2
                               India                            3
                               Isle Bourbon & Isle of France    5
                               Louisiana                        1
                               Madagascar                       2
                               New France                       2
                               Senegal                          2
                               Senegal                          1
India                          Isle Bourbon & Isle of France    1
                               Louisiana                        1
Isle Bourbon & Isle of France  Caribbean                        1
                               India                            1
                               Louisiana                        1
Louisiana                      Caribbean                        3
               

In [95]:
total_crossing = 3+5+2+1+1+1+2+1+1+1+1+1
print(f"{total_crossing} influences are crossing the IO/Atlantic boundary")

20 influences are crossing the IO/Atlantic boundary
